In [1]:
import pandas as pd
import os
from tqdm.notebook import tqdm
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

tags = {
    "loss": "Loss_MB/train_phase/train_stream/Task000",
    "final_accuracy": "Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000",
}

def load_event_file(experiment_directory: str) -> EventAccumulator:
    """
    Loads the event file from a given experiment directory.
    """
    ea: EventAccumulator = EventAccumulator(experiment_directory)
    ea.Reload()
    return ea

def final_metrics(ea: EventAccumulator) -> dict:
    """
    Returns the final metrics from the event file.
    """
    result = {}
    for tag_name, tag in tags.items():
        if tag in ea.Tags()['scalars']:
            result[tag_name] = ea.Scalars(tag)[-1].value
    return result



In [2]:
experiment_logs = "experiment_logs"

records = []

# Loop over each directory in a directory
for experiment in tqdm(os.listdir(os.path.join(experiment_logs))):
    i, dataset, arch, strategy = experiment.split("_")

    record = {
        "dataset": dataset,
        "architecture": arch,
        "strategy": strategy,
        "id": i,
    }

    try:
        ea = load_event_file(os.path.join(experiment_logs, experiment))
        record.update(final_metrics(ea))
    except KeyError as e:
        print(f"Could not load required metric {e} in '{experiment}'")

    records.append(record)

df = pd.DataFrame.from_dict(records)
df

  0%|          | 0/39 [00:00<?, ?it/s]

,dataset,architecture,strategy,id,loss,final_accuracy
0,splitCORe50,VAE,cumulative,0021,0.321811,0.360358
1,splitCIFAR10,AE,cumulative,0015,0.286854,0.872100
2,splitCIFAR10,AE,taskOracle,0032,0.422142,0.937500
3,splitCIFAR100,VAE,taskOracle,0037,1.664222,0.591000
4,splitCIFAR100,AE,taskOracle,0033,1.803951,0.547500
5,splitFMNIST,VAE,taskInference,0043,0.252686,0.992700
6,splitFMNIST,VAE,taskInference,0006,NaN,NaN
7,splitCIFAR10,VAE,taskInference,0044,0.274054,0.729500
8,splitCIFAR100,AE,cumulative,0016,0.384062,0.616700
9,splitFMNIST,AE,cumulative,0001,NaN,NaN


In [3]:
df.dropna().sort_values(by='id').groupby(['dataset', 'architecture', 'strategy']).max().pivot_table(index=['architecture', 'strategy'], columns=['dataset'], values='final_accuracy').to_csv('accuracy.csv')

In [4]:
df.to_csv("results/results.csv")

In [28]:

for strategy in ["cumulative", "taskOracle", "finetuning", "taskInference"]:
    for arch in ["AE", "VAE"]:
        for dataset in [
            "splitFMNIST",
            "splitCIFAR10",
            "splitCIFAR100",
            "splitCORe50",
            "splitEmbeddedCIFAR100",
            "splitEmbeddedCORe50"]:
            try:
                result = df[df['dataset'] == dataset]
                result = result[result['strategy'] == strategy]
                result = result[result['architecture'] == arch]
                accuracy = result.sort_values(by='id').max()["final_accuracy"]
                # accuracy = result.sort_values("id")[-1]["accuracy"]
                print(f"{accuracy:0.4f}", end="\t")
            except:
                continue
        print()

0.9151	0.8721	0.6167	0.3560	nan	nan	
0.8989	0.8725	0.6106	0.3604	nan	nan	
0.9938	0.9375	0.5475	0.4322	nan	nan	
0.9936	0.9382	0.5910	0.4032	nan	nan	
0.1995	0.1939	0.0899	0.0745	nan	nan	
0.1995	0.1945	0.0891	0.0751	nan	nan	
0.9597	0.7123	0.2351	0.0553	nan	nan	
0.9927	0.7295	0.3091	0.0531	nan	nan	
